# 🎤 Preparazione Dataset per Piper TTS

Questo notebook prepara il dataset LJSpeech-Italian per training con Piper TTS.

## 📊 Dataset Disponibili:

### 1. z-uo/female-LJSpeech-italian (CONSIGLIATO per call center)
- 🎤 Voce femminile
- ⏱️ Durata: 8h 23m
- 🔊 Sample rate: 16kHz
- 📦 ~600 MB

### 2. z-uo/male-LJSpeech-italian
- 🎤 Voce maschile
- ⏱️ Durata: 31h 45m
- 🔊 Sample rate: 16kHz
- 📦 ~2.5 GB

## ⚙️ Setup:
1. Runtime → Change runtime type → **GPU T4**
2. Monta Google Drive
3. Scegli dataset (female/male) nella cella di download
4. Esegui celle in ordine

## 1️⃣ Setup Ambiente

In [ ]:
# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Installa dipendenze
!pip install -q datasets soundfile tqdm torchcodec
print("✅ Installazione completata!")

## 2️⃣ Download LJSpeech-Italian

In [ ]:
# Verifica RAM e GPU disponibili
import psutil
import torch

# RAM
ram_total = psutil.virtual_memory().total / 1e9
ram_available = psutil.virtual_memory().available / 1e9
ram_percent = psutil.virtual_memory().percent

print("💾 MEMORIA RAM:")
print(f"   Totale: {ram_total:.1f} GB")
print(f"   Disponibile: {ram_available:.1f} GB")
print(f"   Uso: {ram_percent:.1f}%")

if ram_available < 5:
    print("\n⚠️  RAM BASSA! Raccomandazioni:")
    print("   - Usa MAX_SAMPLES=500 per test")
    print("   - Riavvia runtime (Runtime → Restart runtime)")
    print("   - Chiudi altre tab/notebook Colab")
else:
    print("\n✅ RAM sufficiente per dataset completo")

# GPU (se disponibile)
print("\n🎮 GPU:")
if torch.cuda.is_available():
    print(f"   Disponibile: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("   ⚠️  GPU non disponibile (non necessaria per download)")

# Disco
disk = psutil.disk_usage('/content')
print(f"\n💿 DISCO:")
print(f"   Libero: {disk.free / 1e9:.1f} GB")

if disk.free / 1e9 < 10:
    print("   ⚠️  Spazio limitato! Dataset potrebbe non completarsi")

## 🔍 Verifica Risorse (Opzionale ma Consigliato)

In [ ]:
# ============================================================
#  DOWNLOAD LJSPEECH-IT - VERSIONE ULTRA-OTTIMIZZATA
# ============================================================

print("="*60)
print("  DOWNLOAD LJSPEECH-IT - MEMORY-SAFE MODE")
print("="*60)
print()

import os
from datasets import load_dataset
from tqdm import tqdm
import soundfile as sf
import gc  # Garbage collector per liberare memoria
import psutil
import time

# Installa torchcodec se mancante
try:
    import torchcodec
except ImportError:
    print("📦 Installazione torchcodec...")
    !pip install -q torchcodec
    print("✅ torchcodec installato")

# ============================================================
# ⚙️  CONFIGURAZIONE DOWNLOAD
# ============================================================

# Scegli dataset
DATASET_NAME = "z-uo/female-LJSpeech-italian"  # 5856 file, 8h 23m, voce femminile
# DATASET_NAME = "z-uo/male-LJSpeech-italian"  # 13k+ file, 31h 45m, voce maschile

# ⚠️  IMPORTANTE: Limita numero di sample
# MAX_SAMPLES = None        # ✅ SCARICA TUTTI I FILE (5856 per female)
# MAX_SAMPLES = 100         # 🧪 Test rapido (100 file, ~2 minuti)
# MAX_SAMPLES = 500         # 🧪 Test medio (500 file, ~10 minuti)
# MAX_SAMPLES = 1000        # 🧪 Test grande (1000 file, ~20 minuti)

MAX_SAMPLES = 1000  # ⬅️ MODIFICA QUI! (None = TUTTI I FILE)

# Configurazione memoria
BATCH_SIZE = 50  # Salva metadata ogni N file (ridotto da 100)
MIN_RAM_GB = 2.0  # Ferma download se RAM disponibile < 2GB
CHECK_MEMORY_EVERY = 10  # Controlla memoria ogni N file

# ============================================================

if MAX_SAMPLES is None:
    print("⚠️  MODALITÀ: DOWNLOAD COMPLETO")
    print(f"   Verranno scaricati TUTTI i file del dataset")
    print(f"   Tempo stimato: ~2 ore per female, ~8 ore per male")
    print()
else:
    print(f"⚠️  MODALITÀ: DOWNLOAD LIMITATO")
    print(f"   Verranno scaricati solo {MAX_SAMPLES} file")
    print(f"   Tempo stimato: ~{MAX_SAMPLES * 0.8 / 60:.0f} minuti")
    print()

print(f"📥 Scaricamento dataset: {DATASET_NAME}...")
print("💡 Uso streaming mode + batch processing per sicurezza memoria")
print(f"💾 Batch size: {BATCH_SIZE} file")
print(f"⚠️  Stop automatico se RAM < {MIN_RAM_GB} GB\n")

# Funzione per controllare memoria
def check_memory():
    ram_available = psutil.virtual_memory().available / 1e9
    ram_percent = psutil.virtual_memory().percent
    return ram_available, ram_percent

# Crea directory su Drive
dataset_dir = "/content/drive/MyDrive/piper_training/dataset/ljspeech_italian"
wavs_dir = os.path.join(dataset_dir, "wavs")
metadata_path = os.path.join(dataset_dir, "metadata.csv")
os.makedirs(wavs_dir, exist_ok=True)

# Carica metadata esistente se presente (per resume)
existing_files = set()
if os.path.exists(metadata_path):
    print("📂 Trovato metadata esistente, riprendo download...")
    with open(metadata_path, 'r', encoding='utf-8') as f:
        existing_files = {line.split('|')[0] for line in f.read().strip().split('\n') if line}
    print(f"   File già scaricati: {len(existing_files)}")

print(f"\n💾 Salvataggio su Google Drive...")
print(f"📁 Percorso: {dataset_dir}\n")

# Carica in STREAMING MODE
dataset = load_dataset(DATASET_NAME, split="train", streaming=True)

# Prepara metadata
metadata_lines = []
processed = 0
skipped = 0
stopped_by_memory = False

# Processa sample uno alla volta (streaming)
pbar = tqdm(desc="Salvando audio", unit=" files")

for idx, item in enumerate(dataset):
    try:
        # Limita numero di sample se richiesto
        if MAX_SAMPLES and idx >= MAX_SAMPLES:
            print(f"\n⏸️  Limite raggiunto: {MAX_SAMPLES} sample processati")
            break

        # Controllo memoria ogni N file
        if processed % CHECK_MEMORY_EVERY == 0:
            ram_available, ram_percent = check_memory()

            if ram_available < MIN_RAM_GB:
                print(f"\n⚠️  MEMORIA BASSA! Disponibili: {ram_available:.2f} GB")
                print(f"   Stop automatico per sicurezza. File salvati: {processed}")
                stopped_by_memory = True
                break

            # Aggiorna barra progresso con info memoria
            pbar.set_postfix({
                'RAM': f'{ram_available:.1f}GB',
                'processed': processed,
                'skipped': skipped
            })

        # Nome file
        audio_filename = f"LJ_{idx:06d}.wav"

        # Skip se già esiste
        if audio_filename in existing_files:
            skipped += 1
            pbar.update(1)
            continue

        # Salva audio
        audio_path = os.path.join(wavs_dir, audio_filename)

        # Estrai audio array e sample rate
        audio_data = item['audio']
        sf.write(audio_path, audio_data['array'], audio_data['sampling_rate'])

        # Aggiungi a metadata buffer
        text = item['text']
        metadata_lines.append(f"{audio_filename}|{text}")

        processed += 1
        pbar.update(1)

        # Salva metadata e libera memoria ogni BATCH_SIZE file
        if processed % BATCH_SIZE == 0:
            # Append metadata (non sovrascrive)
            mode = 'a' if os.path.exists(metadata_path) else 'w'
            with open(metadata_path, mode, encoding='utf-8') as f:
                f.write('\n'.join(metadata_lines) + '\n')

            # Pulisci buffer
            metadata_lines = []

            # Garbage collection aggressivo
            gc.collect()

            # Pausa breve per liberare risorse
            time.sleep(0.1)

    except MemoryError:
        print(f"\n❌ ERRORE MEMORIA al file {idx}!")
        print(f"   File salvati con successo: {processed}")
        stopped_by_memory = True
        break
    except Exception as e:
        print(f"\n⚠️  Errore sample {idx}: {e}")
        continue

pbar.close()

# Salva metadata rimanente
if metadata_lines:
    mode = 'a' if os.path.exists(metadata_path) else 'w'
    with open(metadata_path, mode, encoding='utf-8') as f:
        f.write('\n'.join(metadata_lines) + '\n')

# Cleanup
del dataset
del metadata_lines
gc.collect()

# Report finale
print(f"\n{'⚠️  DOWNLOAD PARZIALE' if stopped_by_memory else '✅ Completato!'}")
print(f"   📊 {processed} file nuovi salvati")
if skipped > 0:
    print(f"   ⏭️  {skipped} file già esistenti (saltati)")
print(f"   📄 metadata.csv aggiornato")
print(f"   💾 Totale: ~{processed * 0.5:.1f} MB")
print(f"\n📋 Dataset usato: {DATASET_NAME}")
print(f"   🎤 Tipo: {'Voce femminile' if 'female' in DATASET_NAME else 'Voce maschile'}")

if stopped_by_memory:
    print(f"\n💡 SUGGERIMENTI:")
    print(f"   1. Riavvia runtime: Runtime → Restart runtime")
    print(f"   2. Riesegui questa cella: riprenderà da dove si è fermato")
    print(f"   3. Oppure riduci MAX_SAMPLES")

if MAX_SAMPLES and not stopped_by_memory:
    print(f"\n⚠️  ATTENZIONE: Download limitato a {processed} file su {5856 if 'female' in DATASET_NAME else 13000}+ disponibili")
    print(f"   Per scaricare tutto: imposta MAX_SAMPLES = None e riesegui")

# Memoria finale
ram_available, ram_percent = check_memory()
print(f"\n💾 RAM finale: {ram_available:.1f} GB disponibili ({100-ram_percent:.1f}% libera)")

## 3️⃣ Verifica Dataset

In [ ]:
# Verifica file creati
import pandas as pd
from IPython.display import Audio, display

dataset_dir = "/content/drive/MyDrive/piper_training/dataset/ljspeech_italian"
metadata_path = os.path.join(dataset_dir, "metadata.csv")

# Carica metadata
df = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
print(f"📊 Dataset info:")
print(f"   File audio: {len(df)}")
print(f"   Testo totale: {df['text'].str.len().sum()} caratteri")
print(f"   Testo medio: {df['text'].str.len().mean():.1f} caratteri/sample")

# Test audio casuale
sample_idx = 0
sample_row = df.iloc[sample_idx]
sample_audio = os.path.join(dataset_dir, "wavs", sample_row['filename'])

print(f"\n🎤 Sample test #{sample_idx}:")
print(f"   Testo: {sample_row['text']}")
print(f"   File: {sample_row['filename']}")

display(Audio(sample_audio))

## 4️⃣ Statistiche Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Lunghezza testi
text_lengths = df['text'].str.len()

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(text_lengths, bins=50, edgecolor='black')
plt.xlabel('Lunghezza testo (caratteri)')
plt.ylabel('Frequenza')
plt.title('Distribuzione lunghezza testi')
plt.axvline(text_lengths.mean(), color='r', linestyle='--', label=f'Media: {text_lengths.mean():.1f}')
plt.legend()

plt.subplot(1, 2, 2)
word_counts = df['text'].str.split().str.len()
plt.hist(word_counts, bins=50, edgecolor='black')
plt.xlabel('Numero parole')
plt.ylabel('Frequenza')
plt.title('Distribuzione numero parole')
plt.axvline(word_counts.mean(), color='r', linestyle='--', label=f'Media: {word_counts.mean():.1f}')
plt.legend()

plt.tight_layout()
plt.show()

print("📈 Statistiche:")
print(f"   Lunghezza testo: {text_lengths.min()}-{text_lengths.max()} caratteri")
print(f"   Parole per sample: {word_counts.min()}-{word_counts.max()} parole")
print(f"   Totale parole: {word_counts.sum():.0f}")

## ✅ Dataset Pronto!

Il dataset è ora pronto per training con Piper TTS.

### Prossimi passi:
1. Segui la guida ufficiale Piper: https://github.com/rhasspy/piper
2. Configura training con il dataset preparato
3. Avvia training (richiede diverse ore su GPU)

### Note:
- Dataset: `/content/drive/MyDrive/piper_training/dataset/ljspeech_italian`
- Metadata: `metadata.csv` (formato: filename|text)
- Audio: cartella `wavs/` (formato WAV)